Copyright (C) 2023 Robin Lamberti.

This file is part of german-mop-vote-behaviour.

german-mop-vote-behaviour is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

german-mop-vote-behaviour is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with german-mop-vote-behaviour. If not, see <http://www.gnu.org/licenses/>.

In [1]:
pip install scikit-network

In [2]:
import numpy as np
import json

In [5]:
members = np.array(json.load(open('../../members.json')))

In [8]:
attraction = np.array(json.load(open('../../attraction.json')))

In [9]:
from_nodes = [edge["a"] for edge in attraction]

In [10]:
to_nodes = [edge["b"] for edge in attraction]

In [11]:
weights = [edge["attraction"] for edge in attraction]

In [12]:
unique_nodes = np.unique(np.concatenate((from_nodes, to_nodes)))

In [13]:
name_to_party = {member["name"]: member["party"] for member in members}
parties = [name_to_party[node_name] for node_name in unique_nodes]

In [14]:
num_nodes = len(unique_nodes)

In [15]:
adjacency_matrix = np.zeros((num_nodes, num_nodes), dtype=int)

In [16]:
for i in range(len(attraction)):
    from_index = np.where(unique_nodes == from_nodes[i])[0][0]
    to_index = np.where(unique_nodes == to_nodes[i])[0][0]
    adjacency_matrix[from_index, to_index] = weights[i]

In [17]:
adjacency_matrix

array([[  0,   0,   0, ...,   0, 170, 158],
       [166,   0, 104, ..., 170, 206, 194],
       [ 78,   0,   0, ...,   0, 102, 100],
       ...,
       [142,   0,  82, ...,   0, 176, 162],
       [  0,   0,   0, ...,   0,   0, 196],
       [  0,   0,   0, ...,   0,   0,   0]])

In [18]:
from scipy import sparse

In [19]:
adjacency_matrix = sparse.csr_matrix(adjacency_matrix)

In [20]:
adjacency_matrix

<752x752 sparse matrix of type '<class 'numpy.intc'>'
	with 281183 stored elements in Compressed Sparse Row format>

In [21]:
from IPython.display import SVG
from sknetwork.visualization import svg_graph

In [22]:
from sknetwork.embedding import Spring

In [24]:
colors = ['Die Linke', 'B90/GrÃ¼ne', 'SPD', 'FDP', 'CDU/CSU', 'AfD', 'fraktionslos']
labels = [colors.index(f) for f in parties]
label_colors = ['pink', 'green', 'red', 'yellow', 'black', 'blue', 'white']

In [25]:
init_pos = {(x, y): np.array([[e / x, np.random.randn() / y] for e in labels]) for y in [1.3] for x in [8]}

In [26]:
spring = Spring(2)
embedding = {(x, y, i): spring.fit_transform(adjacency_matrix, n_iter = i, position_init = pos) for (x ,y), pos in init_pos.items() for i in [10]}

In [27]:
embedding_freely = {n: spring.fit_transform(adjacency_matrix, n_iter = n) for n in [80, 100, 105, 120, 130, 140]}

In [28]:
image = [svg_graph(adjacency_matrix, position=e, display_edges=False, scale=10, name_position='below', names=unique_nodes, labels=labels, label_colors=label_colors, filename=f'out\\attraction_{x}_{y:.1f}_{i}') for (x, y, i), e in embedding.items()]
image = [svg_graph(adjacency_matrix, position=e, display_edges=False, scale=10, name_position='below', names=unique_nodes, labels=labels, label_colors=label_colors, filename=f'out\\attraction_freely_{i}') for i, e in embedding_freely.items()]